In [1]:
import numpy as np
from scipy.linalg import expm
import math
from tqdm import tqdm
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import expm_multiply
import torch
from time import time

In [2]:
def level_fct(y): # We assume CPMM
        return (10**8) / y

In [3]:
def _calculate_matrix_t(y_grid, St):
    nsims, Nt_plus1 = St.shape
    ny = len(y_grid)
    A_matrix = np.zeros((ny, ny, nsims, Nt_plus1))
    
    for i in range(ny):
        quantity = y_grid[i]
        
        if i < ny - 1:
            quantity_P1 = y_grid[i+1]
            diff = 2 * (level_fct(quantity) - level_fct(quantity_P1))
            A_matrix[i, i+1] = 100 * np.exp(-St - 1 + diff)
        
        if i > 0:
            quantity_M1 = y_grid[i-1]
            diff = -2 * (level_fct(quantity_M1) - level_fct(quantity))
            A_matrix[i, i-1] = 100 * np.exp(St - 1 + diff)
    
    return A_matrix

In [4]:
def _calculate_omega_pytorcht(y_grid, t, St):
    import numpy as np
    import torch
    from time import time

    A_np = _calculate_matrix_t(y_grid, St)  # shape: (dim, dim, nsims, Nt+1)

    # Move axes: shape (nsims, Nt+1, dim, dim)
    A_batch = np.moveaxis(A_np, [2, 3], [0, 1])

    # Convert to torch tensor
    A = torch.from_numpy(A_batch).double()  # shape: (nsims, Nt+1, dim, dim)
    nsims, Nt_plus1, dim, _ = A.shape

    dt = 1 - t  # assumed scalar; update here if this varies

    vec = torch.ones((dim, 1), dtype=torch.double)

    omega = torch.zeros((nsims, Nt_plus1, dim), dtype=torch.double)

    init_time = time()
    for i in range(nsims):
        for j in range(Nt_plus1):
            Aij = A[i, j] * dt  # shape: (dim, dim)
            try:
                expAij = torch.matrix_exp(Aij)  # shape: (dim, dim)
                omega[i, j] = (expAij @ vec).squeeze(-1)
            except RuntimeError as e:
                print(f"Matrix exp failed at sim {i}, t {j}: {e}")
                omega[i, j] = torch.nan
    end_time = time()

    print(f"Total execution time: {end_time - init_time}")
    return omega.numpy()

In [5]:
y_grid = np.array([990 + i*0.5 for i in range(0,41)])

In [6]:
nsims = 1_000
Nt = 1000
timesteps = np.linspace(0, 1, Nt+1)

In [7]:
dW = np.random.normal(0, np.sqrt(1/Nt), (nsims, Nt))
W = np.concatenate([np.zeros((nsims, 1)), np.cumsum(dW, axis=1)], axis=1)  # shape (nsims, Nt+1)
St = 100 + 0.2 * W  # shape (nsims, Nt+1)

In [8]:
_calculate_omega_t(y_grid,0.5,St)

: 

: 